# Benzinga-Nachrichten-Verarbeitung

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cwd = "/content/drive/MyDrive/NewsTrading/trading_bot"
%cd /content/drive/MyDrive/NewsTrading/trading_bot

/content/drive/MyDrive/NewsTrading/trading_bot


In [3]:
!sudo apt update
!sudo apt install maven

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,283 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 1,621 kB in 2s (1,063 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
18 packages can be u

In [4]:
%%capture
!pip install html2text
!pip install datefinder
!pip install -U dask[complete]
!pip install nltk
!pip install dateparser
!pip install sutime;

In [5]:
!mvn dependency:copy-dependencies -DoutputDirectory=./jars -f $(python3 -c 'import importlib; import pathlib; print(pathlib.Path(importlib.util.find_spec("sutime").origin).parent / "pom.xml")')


[INFO] Scanning for projects...
[INFO] 
[INFO] ----------< edu.stanford.nlp:stanford-corenlp-sutime-python >-----------
[INFO] Building CoreNLP SUTime Python Wrapper 1.4.0
[INFO] --------------------------------[ jar ]---------------------------------
[INFO] 
[INFO] --- maven-dependency-plugin:3.1.2:copy-dependencies (default-cli) @ stanford-corenlp-sutime-python ---
[INFO] javax.xml.bind:jaxb-api:jar:2.4.0-b180830.0359 already exists in destination.
[INFO] javax.activation:javax.activation-api:jar:1.2.0 already exists in destination.
[INFO] com.sun.xml.bind:jaxb-core:jar:2.3.0.1 already exists in destination.
[INFO] com.sun.xml.bind:jaxb-impl:jar:2.4.0-b180830.0438 already exists in destination.
[INFO] javax.activation:activation:jar:1.1.1 already exists in destination.
[INFO] edu.stanford.nlp:stanford-corenlp:jar:4.0.0 already exists in destination.
[INFO] com.apple:AppleJavaExtensions:jar:1.4 already exists in destination.
[INFO] de.jollyday:jollyday:jar:0.4.9 already exists in dest

In [6]:
%load_ext autoreload
%autoreload 2
import dask.dataframe as dd
import dask
import pandas as pd
from dask.distributed import Client
from src.preprocessing.news_parser import filter_body, time, body_formatter, get_company_abbreviation
import re
import plotly.express as px
import nltk
nltk.download('punkt')
import yfinance as yf

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# client = Client(memory_limit='25GB', processes=False,
#                 n_workers=2, threads_per_worker=1)
# client

## HTML-Parsing
Als erstes müssen wir die HTML-Dokumente zu normalem Text umwandeln, ansonsten sind die Text-Zellen zu groß und führen zu Problemen mit PyArrow/Dask.

In [ ]:
dask.config.set(scheduler="threads")

In [ ]:
input_dir = "data/raw_bzg/"
output_dir = 'data/unraw1_bzg/'

In [ ]:
# for year in range(2019, 2020):
#     print(year)
#     df = pd.read_parquet(f"{input_dir}story_df_raw_{year}.parquet")
#     df = dd.from_pandas(df, npartitions=12)
#     df["html_body"] = df["html_body"].apply(body_formatter, meta=pd.Series(dtype="str"))
#     df = df.rename(columns={"html_body":"body"})
#     name_function = lambda x: f"data-{year}-{x}.parquet"
#     df.to_parquet(output_dir, name_function=name_function)

## Neu-Partitionierug
Sodass alle Partitionen etwa die gleiche Größe haben.

In [ ]:
input_dir = 'data/unraw1_bzg/'
output_dir = 'data/unraw2_bzg/'

# ddf = dd.read_parquet(input_dir+"*.parquet")
# ddf2 = ddf.repartition(npartitions=50)
# name_function = lambda x: f"data-{x}.parquet"
# ddf2.to_parquet(output_dir, name_function=name_function)

## Author-Inferenz

Ein bisschen die Daten säubern...

In [ ]:
input_dir = cwd+'/data/unraw2_bzg/'
output_dir = cwd+'/data/unraw3_bzg/'

In [ ]:
ddf = dd.read_parquet(input_dir+"*.parquet")

In [ ]:
# Remove rows for which noo stock ticker is recorded
ddf = ddf[ddf.stocks != '']

In [ ]:
# Convert `channels`  datatype from string to list
ddf["channels"] = ddf["channels"].apply(eval, meta=pd.Series(dtype='object'))

Untersuche als nächstes die Behauptung, dass **PRNewswire** und **Businesswire** den gesamten Markt für Pressemeldungen in den USA kontrollieren. Wenn dem so ist, und sie nicht noch weitere, unwichtige Meldungen veröffentlichen, dann können wir einfach die Newsartikel nach diesen Autoren filtern und uns viel Arbeit ersparen.

In [ ]:
dask.config.set(scheduler="processes")
ddf["inferred_author"] = None

def infer_author(body):
  for author in ["PRNewswire", "Globe Newswire", "Business Wire", "ACCESSWIRE"]:
    if re.search(author, body, re.IGNORECASE) is not None:
      return author
  return None

ddf["inferred_author"] = ddf.body.apply(infer_author, meta=pd.Series(dtype="string"))

In [ ]:
# value_counts for authors
auhtor_value_counts = pd.concat([ddf.author.value_counts().head(10), ddf.inferred_author.value_counts().head(10)], axis=1)

In [ ]:
auhtor_value_counts

,author,inferred_author
Benzinga,1061214,NaN
PRNewswire,305720,587242.0
Globe Newswire,293466,475171.0
Business Wire,268561,293052.0
Newsfile,70877,NaN
ACCESSWIRE,62615,81054.0
"AB Digital, Inc.",9936,NaN
WebWire,6404,NaN
PRWeb,2617,NaN
News Direct,2080,NaN


In [ ]:
auhtor_value_counts.sum().diff()

author                  NaN
inferred_author   -646971.0
dtype: float64

Ungefähr 650k Nachrichten werden ausgelassen, wenn nur die vier Hauptvertreiber von Pressemeldungen berücksichtigt werden.

In [ ]:
ddf = ddf[~ddf.inferred_author.isna()]

In [ ]:
ddf["inferred_author"] = ddf["inferred_author"].astype("string")

In [ ]:
ddf["channels"] = ddf.channels.apply(lambda x: str(x), meta=pd.Series(dtype="string"))

In [ ]:
ddf.inferred_author.value_counts().compute()

PRNewswire        587242
Globe Newswire    475171
Business Wire     293052
ACCESSWIRE         81054
Name: inferred_author, dtype: Int64

In [ ]:
ddf.inferred_author.value_counts().sum().compute()

1436519

In [ ]:
name_function = lambda x: f"data-{x}.parquet"
ddf.to_parquet(output_dir, name_function=name_function)

In [ ]:
# Contains 100k rows
earnings_ddf = ddf[ddf.channels.apply(lambda x: "Earnings" in x, meta=pd.Series(dtype=bool))]

In [ ]:
# value counts for authors of earnings reports (contrast to value counts of all news articles)
earnings_ddf.inferred_author.value_counts().head(10)

Globe Newswire    44589
PRNewswire        31440
ACCESSWIRE        16434
Name: inferred_author, dtype: Int64

Hier sehen wir, dass es keine einzige Pressemeldung von **Business Wire** gibt, die mit *Earnings* gekennzeichnet sind. Trotzdem gibt es relevante *Earnings* reports von Business Wire. Dies habe ich kurz verifiziert...

## Russell-Filtering und Ticker-Namen-Mapping

Wie viele Nachrichten bleiben, wenn wir auch die momentane Zusammensetzung des Russell 3000 filtern?

In [7]:
input_dir = cwd+'/data/unraw3_bzg/'
output_dir = cwd+'/data/unraw4_bzg/'
ddf = dd.read_parquet(input_dir)

In [8]:
# Around 3k tickers at this moment
russell_tickers = pd.read_pickle("data/tickers.pkl")
russell_tickers = russell_tickers.categories

In [ ]:
# Get company name by ticker (longName is always equal to shortName in yf...)
# This takes a long time, because of the api calls to yf (15min)
def yahoo_get_wrapper(x):
  try:
    return yf.Ticker(x).info.get("longName")
  except:
    return None

company_names = pd.Series(russell_tickers).apply(lambda x: yahoo_get_wrapper(x))

In [ ]:
mapper = pd.concat([company_names, pd.Series(russell_tickers)], axis=1)

In [ ]:
mapper.columns = ["company_names", "ticker"]
mapper = mapper[mapper.isna().sum(axis=1) == 0]

In [ ]:
mapper = mapper.set_index("ticker")

In [ ]:
company_endings = pd.read_table("data_shared/corporation_endings.txt").iloc[:, 0]
mapper["short_name"] = mapper.company_names.apply(lambda x: get_company_abbreviation(x,
                                                                                     company_endings=company_endings))

In [ ]:
print(mapper.short_name.isna().sum()) # 84 stocks for which we don't have an ending to abbreviate
mapper.loc[:, "short_name"] = mapper.short_name.fillna(mapper.company_names)


83

In [ ]:
mapper.to_parquet(cwd + "/data_shared/ticker_name_mapper.parquet")

In [9]:
mapper = pd.read_parquet(cwd + "/data_shared/ticker_name_mapper.parquet")

In [10]:
filt_ddf = ddf[ddf.stocks.isin(mapper.index.to_list())]

In [11]:
# ddf.shape[0].compute()

In [12]:
# filt_ddf.shape[0].compute()

Es verbleiben circa 660k Nachrichten, die für unser Russell 3000-Aktienuniversum relevant sind. Dieses Filtering hätten wir eigentlich auch schon früher machen können... Aber egal.

In [13]:
filt_ddf = filt_ddf.set_index("time")

In [14]:
monthly_news_counts = filt_ddf.stocks.resample("MS").count()

In [15]:
# px.line(monthly_news_counts.compute(), title="# of articles per month")

In [16]:
dask.config.set(scheduler="processes")

Diese Nachrichten können wir nun wirklich parsen, und danach ordentlich kategorisieren.

In [17]:
ddf = filt_ddf
ddf = ddf.drop(columns=["author"]).rename(columns={"inferred_author":"author"})

In [18]:
ddf["company_name"] = ddf.stocks.apply(lambda x: mapper.company_names.loc[x], meta=pd.Series(dtype="string"))
ddf["short_name"] = ddf.stocks.apply(lambda x: mapper.short_name.loc[x], meta=pd.Series(dtype="string"))

In [19]:
name_function = lambda x: f"data-{x}.parquet"
ddf.to_parquet(cwd+'/data/latest/', name_function=name_function)

## Parsing und Klassifikation von Nachrichten


In [20]:
ddf = dd.read_parquet(cwd+'/data/latest/')

In [21]:
sample_partition = ddf.get_partition(40)

In [22]:
x = sample_partition.head(5).iloc[3]
x

stocks                                                       YELL
title           Ecobat Completes Leadership Team with Addition...
channels                        ['Commodities', 'Press Releases']
body            IRVING, Texas, Aug. 9, 2021 /PRNewswire/ -- Ec...
author                                                 PRNewswire
company_name                                   Yellow Corporation
short_name                                                Yellow 
Name: 2021-08-09 13:04:00-04:00, dtype: object

In [23]:
x.body

'IRVING, Texas, Aug. 9, 2021 /PRNewswire/ -- Ecobat, the global leader in\nrecycling batteries, announced today the appointment of Jamie Pierson as chief\nfinancial officer.\n\nJamie was most recently CFO for YRC Worldwide (NASDAQ:YELL), where he was\nresponsible for operational and financial strategy for the company. In\naddition to his financial responsibilities, Jamie managed relationships with\ncredit rating agencies, financial and industry analysts, lenders and\nshareholders. Jamie has over 10 years of investment banking, financial\nrestructuring, advisory and corporate development experience including:\nstrategic planning, M&A advisory, capital raising and business consulting\nservices. His past experiences also include developing the acquisition and\nintegration process for a $1.1B logistics holding company. Before YRC, Jamie\nworked at Alvarez & Marsal, Prime Source Building Products Inc, and Horizon\nGlobal Corporation as interim CFO. Jamie holds a BBA in Finance with a\nconce

In [27]:
x.company_name

'Yellow Corporation'

In [24]:
filter_body(x.body, x.stocks, x.author, x.name, x.company_name, x.short_name) # x.name == time

'Ecobat, the global leader in recycling batteries, announced today the appointment of Jamie Pierson as chief financial officer. Jamie was most recently CFO for YRC Worldwide , where he was responsible for operational and financial strategy for the company. In addition to his financial responsibilities, Jamie managed relationships with credit rating agencies, financial and industry analysts, lenders and shareholders. Jamie has over 10 years of investment banking, financial restructuring, advisory and corporate development experience including: strategic planning, M&A advisory, capital raising and business consulting services. His past experiences also include developing the acquisition and integration process for a $1.1B logistics holding company. Before YRC, Jamie worked at Alvarez & Marsal, Prime Source Building Products Inc, and Horizon Global Corporation as interim CFO. Jamie holds a BBA in Finance with a concentration in accounting and an MBA in Finance and Entrepreneurship from Th

Weiter machen...

In [38]:
LINK_SENTENCE_REGEX = "www\.[a-z]*\.com"
re.sub(f".([^.])*{LINK_SENTENCE_REGEX}([^.]*).?", ".", "Hey. Come visit now at www.klontyke.com")

'Hey.'

In [28]:
sample_partition["parsed_body"] = sample_partition.apply(lambda x: filter_body(x.body,
                                                                               x.stocks,
                                                                               x.author,
                                                                               x.name,
                                                                               x.company_name,
                                                                               x.short_name),
                                                         axis=1,
                                                         meta=pd.Series(dtype="string"))

In [29]:
sample_partition

,stocks,title,channels,body,author,company_name,short_name,parsed_body
npartitions=1,,,,,,,,
,string,string,object,string,string,object,object,string
,...,...,...,...,...,...,...,...


In [30]:
sample_partition.head()

KeyboardInterrupt: ignored